In [1]:
# ВВОД ССЫЛКИ
url = 'https://provagon.com/rolling%20stock'
directurl = 'https://provagon.com/spare%20parts/51F21EDEAB2C5E2FC22583BE005E50BFQ=Tseny-predlozheniya-novykh-gruzovykh-vagonov-predpriyatiyami-iz'

In [2]:
import mechanize
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import Chrome
import time
import locale
from selenium.webdriver.common.action_chains import ActionChains 
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_rows', 500)

C:\Apps\Anaconda\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
#запуск хрома
driver = webdriver.Chrome()
actions = ActionChains(driver)

#переход на сайт
driver.get(url)

#поиск последнего отчета 'Цены предложения новых грузовых вагонов'
driver.find_element_by_partial_link_text("Цены предложения новых грузовых вагонов предприятиями из стран Таможенного союза").click();

#поиск и ввод логина-пароля (работает только для общей ссылки)
time.sleep(1)
login = driver.find_element_by_id("lgn")
actions = ActionChains(driver)
actions.move_to_element(login)
actions.click()
actions.send_keys("gpbank")
actions.perform()

time.sleep(1)
password = driver.find_element_by_id("pw")
actions = ActionChains(driver)
actions.move_to_element(password)
actions.click()
actions.send_keys("UTE597nk")
actions.perform()
time.sleep(1)

# поиск кнопки логина и логин
driver.find_element_by_xpath('//*[@id="id_rcCorner"]/form/center/table/tbody/tr[5]/td[2]/input').click()
#ожидание загрузки
time.sleep(5)

#вывод даты отчета
dateofreport = driver.find_element_by_xpath('//*[@id="id_rcCorner"]/div[2]')
print('Дата отчета:',dateofreport.get_attribute('textContent'))

Дата отчета: 12.04.19


In [4]:
#извлечение таблицы

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
table = soup.find('table')
rows = table.find_all('tr')

#парсинг таблицы, удаление пробелов и трансформация значений в число 
zero_data = np.zeros(shape=(len(rows),4))
df = pd.DataFrame(zero_data)

i=0
j=0
for row in table.find_all('tr'):
    #проверка структуры ячеек. сдвоенные ячейки имеют длину 6, 4 или 2
    if len(row) == 8:
        for cell in row.find_all('td'):
            df.iloc[i,j] = cell.text.replace('\n','')
            j += 1
        i += 1
        j = 0
    elif len(row) == 6:
        df.iloc[i,0] = df.iloc[i-1,0]
        j = 1
        for cell in row.find_all('td'):
            df.iloc[i,j] = cell.text.replace('\n','')
            j += 1
        i += 1
        j = 0
    elif len(row) == 4:
        j=0
        for cell in row.find_all('td'):
            df.iloc[i,j] = cell.text.replace('\n','')
            j += 1
        i += 1
    else:
        j = 1
        for cell in row.find_all('td'):
            df.iloc[i,j] = cell.text.replace('\n','')
            j += 1
        i += 1
        j = 0

        
# заполнение пропусков 
for i in range(0,len(df)):
    #заполнение имен вагонов
    if df.iloc[i,0] == 0:
        df.iloc[i,0] = str(df.iloc[i-1,0])
for i in range(2,len(df)):
        #заполнение цен
    if df.iloc[i,2] == 0:
        df.iloc[i,2] = str(df.iloc[i-1,2])
 

In [5]:
#Обработка таблицы
 
                
i=0
for i in range(1, len(df.iloc[1:,2])+1):
    if ',' in str(df.iloc[i,2]):
        df.iloc[i,2] = float(df.iloc[i,2].replace(',','.'))
    if ' ' in str(df.iloc[i,2]):
        df.iloc[i,2] = float(df.iloc[i,2].replace(' ',''))
        
        
for i in range(1, len(df.iloc[1:,3])+1):
    if ',' in str(df.iloc[i,3]):
        df.iloc[i,3] = float(df.iloc[i,3].replace(',','.'))
    if ' ' in str(df.iloc[i,3]):
        df.iloc[i,3] = float(df.iloc[i,3].replace(' ',''))
    
#удаление последней колонки
df = df.drop([3], axis = 1)
    
#удаление индексов, переименование
df=df.drop([0], axis=0)
df.insert(0, 'Завод', '')
df.columns=['Завод','Производители','Модели','Цена за единицу (без учета НДС)']

#удаление строк с названием стран
df = df.drop(df[df['Модели'] == 'РФ'].index)
df = df.drop(df[df['Модели'] == 'Беларусь'].index)
df = df.drop(df[df['Модели'] == 'Казахстан'].index)
df = df.drop(df[df['Модели'] == 'Грузия'].index)
df = df.drop(df[df['Модели'] == 'Украина'].index)

df.reset_index(drop=True, inplace=True)

#внесение имен заводов в столбец заводов        
row = 0
plantname = df['Модели'][0]
for i in df['Модели']:
    if 'завод' in i or 'DAF' in i:
        plantname = i
    df['Завод'][row] = plantname
    row=row+1

#Удаление названий заводов из таблицы
df = df.drop(df[df['Модели'].str.contains('завод')].index) 
df = df.drop(df[df['Модели'].str.contains('DAF')].index) 
df.reset_index(drop=True, inplace=True)
        
# данные по странам заполняются на основании названий заводов. 
df.insert(0, 'Страна', '')
row = 0
for i in df['Завод']:
    if 'Могилевский' in i or 'Осиповичский' in i:
        country = 'Беларусь'
    elif 'Казахстанская' in i or 'ЗИКСТО' in i:
        country = 'Казахстан'
    elif 'Рустави' in i:
        country = 'Грузия'
    elif 'Крюковский' in i or 'Днепровагонмаш' in i or 'Попаснянский' in i or 'Карпаты' in i or 'Укрспецвагон' in i or 'Стрыйский' in i or 'Кривой' in i or 'Верхнеднепровский' in i or 'Азовобщемаш' in i or 'Стахановский' in i or 'Укрспецвагон' in i:
        country = 'Украина'
    elif 'Ташкент' in i or 'Андижанский' in i:
        country = 'Узбекистан'
    else:
        country = 'Россия'
    df['Страна'][row] = country
    row=row+1        
        
# добавление колонок и названий в соответствии с базой вагонов
df.insert(5, 'Род вагона', '')
df.insert(6, 'Месяц', '')
df.insert(7, 'Год', '')
df.insert(8, 'Инновационность', '')
df.insert(9, 'Сокращенное название (для связки со страной)', '')

# получение даты из отчета и заполнение столбцов
locale.setlocale(locale.LC_ALL, '')
date=dateofreport.get_attribute('textContent')
date = datetime.strptime(date, '%d.%m.%y')
df['Год'] = date.year
df['Месяц'] = date.strftime("%B")

df = df.rename(columns = {'Завод':'Производитель','Производители':'Продукция','Цена за единицу (без учета НДС)':'Цена'})
df.head()

#Очистка колонки Модели от пробелов
while any(df['Модели'].str.contains(' ')) == True:
    df['Модели'] = df['Модели'].str.replace(' ','')   
    
#борьба с проблемой сдвоенных ячеек формата "название//код-код//цена". вторая цена ставновится нулевой. 
#в этом случае цена заполняется на основе предыдущей строки
p=0
for price in df['Цена']:
    if price == 0:
        df['Цена'][p] = df['Цена'][p-1]
        df['Продукция'][p] = df['Продукция'][p-1]
    p+=1

p=0
# for item in df['Продукция']:
#     if item == 0:
#         df['Продукция'][p] = df['Продукция'][p-1]    
#     p+=1
df['Модели'] = df['Модели'].str.replace('-','_')
df
#борьба с проблемой сдвоенных ячеек формата "название-название//код-код//цена". второе название ставновится нулевой. 
#в этом случае цена заполняется на основе предыдущей строки    

,Страна,Производитель,Продукция,Модели,Цена,Род вагона,Месяц,Год,Инновационность,Сокращенное название (для связки со страной)
0,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с разгрузочными люками (с осевой наг...,12_9853,4.41,,Апрель,2019,,
1,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с глухим кузовом (с осевой нагрузкой...,12_9869,4.22,,Апрель,2019,,
2,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 120 м. куб.,...",19_9549,4.84,,Апрель,2019,,
3,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 101 м. куб.,...",19_9870_01,4.84,,Апрель,2019,,
4,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 120 м. ку...,19_9549_03,4.84,,Апрель,2019,,
5,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 101 м. ку...,19_9835_01,4.84,,Апрель,2019,,
6,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-цементовоз (с осевой нагрузкой 25 тс),19_9550_01,4.84,,Апрель,2019,,
7,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",фитинговая 80-футовая платформа (с осевой нагр...,13_6903,4.64,,Апрель,2019,,
8,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*",цистерна для перевозок нефтепродуктов (с осево...,15_9993,4.02,,Апрель,2019,,
9,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*","цистерна для перевозок СУГ (сочлененного типа,...",15_9541_01,9.6,,Апрель,2019,,


In [80]:
str(df.iloc[5-1,2])

'4,220'

In [14]:
# импорт базы с вагонами
wagon = pd.read_excel('Букварь вагонов.xlsx', sheet_name='Словарь (версия 2)', skiprows=0, header=0)

#wagon['Модель'] = wagon['Модель'].str.replace('_','-')
wagon.head()

,№,Модель,Подробное описание,Тип,Краткое описание,Завод,Завод 14 букв,Страна,Холдинг,Инновационность,Проверка на уникальность номера,Unnamed: 11,Нет
0,1.0,094_4086.00.000ТТ,хоппер для перевозок сыпучих грузов,Хоппер прочие,Хоппер,Верхнеднепровский машиностроительный завод,Верхнед,Украина,NaN,Нет,0.0,NaN,NaN
1,2.0,11_6874,"крытый вагон (с объемом кузова 175 м. куб., с ...",Крытый,Крытый,ТихвинСпецМаш,ТихвинС,Россия,NaN,Да,0.0,NaN,NaN
2,3.0,11_280,крытый вагон (с объемом кузова 138 м. куб.),Крытый,Крытый,Алтайвагон,Алтайва,Россия,NaN,Нет,0.0,NaN,NaN
3,4.0,11_280_01,крытый вагон с объемом кузова 138 м. куб.,Крытый,Крытый,Алтайвагон,Алтайва,Россия,NaN,Нет,0.0,NaN,NaN
4,5.0,11_2135_01,крытый вагон (с объемом кузова 160 м. куб.),Крытый,Крытый,Алтайвагон,Алтайва,Россия,NaN,Нет,0.0,NaN,NaN


In [32]:
# создание копии 'df' и заполнение копии данными из датафрейма вагонов 
# проверка на совпадение названия страны в базе вагонов и выгрузке с сайта
# по умолчанию страна заполняется на основе выгрузки с сайта
# при несовпадении страны в выгрузке и в базе выводим предупреждение

tb = df.copy()
row1 = 0
for model in tb['Модели']:
    try:
    #tb['Производитель'][row1] = wagon.loc[(wagon['Модель'] == model) & (wagon['Страна'] == tb['Страна'][row1])]['Завод'].values[0]
        tb['Род вагона'][row1] = wagon.loc[(wagon['Модель'] == model) & (wagon['Страна'] == tb['Страна'][row1])]['Тип'].values[0]
        tb['Инновационность'][row1] = wagon.loc[(wagon['Модель'] == model) & (wagon['Страна'] == tb['Страна'][row1])]['Инновационность'].values[0]
        tb['Продукция'][row1] = wagon.loc[(wagon['Модель'] == model) & (wagon['Страна'] == tb['Страна'][row1])]['Подробное описание'].values[0]
    
        row1 += 1
    except Exception:
        print ('возможна проблема в строке: ', row1)
        row1 += 1
# вешаем ярлыки из 7 знаков    
tb['Сокращенное название (для связки со страной)'] = tb['Производитель'].str[0:7]
tb['Сокращенное название (для связки со страной)'] = tb['Сокращенное название (для связки со страной)'].str.upper()


 

C:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [35]:
# сведение месяцев и годов в один столбец, удаление старых
tb['Дата'] = tb['Месяц'].str[0:3] + '.' + tb['Год'].astype(str)

tb = tb.drop(['Месяц', 'Год'], axis = 1)
tb

,Страна,Производитель,Продукция,Модели,Цена,Род вагона,Инновационность,Сокращенное название (для связки со страной),Дата
0,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с разгрузочными люками (с осевой наг...,12_9853,4.41,Полувагон,Да,ТИХВИНС,Апр.2019
1,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с глухим кузовом (с осевой нагрузкой...,12_9869,4.22,Полувагон,Да,ТИХВИНС,Апр.2019
2,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 120 м. куб.,...",19_9549,4.84,Хоппер-зерновоз,Да,ТИХВИНС,Апр.2019
3,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 101 м. куб.,...",19_9870_01,4.84,Хоппер-зерновоз,Да,ТИХВИНС,Апр.2019
4,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 120 м. ку...,19_9549_03,4.84,Хоппер-минераловоз,Да,ТИХВИНС,Апр.2019
5,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 101 м. куб.),19_9835_01,4.84,Хоппер-минераловоз,Да,ТИХВИНС,Апр.2019
6,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-цементовоз (с осевой нагрузкой 25 тс),19_9550_01,4.84,хоппер-цементовоз,Да,ТИХВИНС,Апр.2019
7,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",фитинговая 80-футовая платформа (с осевой нагр...,13_6903,4.64,Платформа фитинговая,Да,ТИХВИНС,Апр.2019
8,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*",цистерна для перевозок нефтепродуктов (с осево...,15_9993,4.02,Цистерна д/нефтепродуктов,Да,ТИХВИНХ,Апр.2019
9,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*","цистерна для перевозок СУГ (сочлененного типа,...",15_9541_01,9.6,Цистерна СУГ. Сочлен,Да,ТИХВИНХ,Апр.2019


In [36]:
tb.to_excel('provagon_output_' + df['Месяц'][0] + '.xlsx', sheet_name = 'sheet0')

,Страна,Производитель,Продукция,Модели,Цена,Род вагона,Инновационность,Сокращенное название (для связки со страной),Дата
0,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с разгрузочными люками (с осевой наг...,12_9853,4.41,Полувагон,Да,ТИХВИНС,Апрель 2019
1,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с глухим кузовом (с осевой нагрузкой...,12_9869,4.22,Полувагон,Да,ТИХВИНС,Апрель 2019
2,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 120 м. куб.,...",19_9549,4.84,Хоппер-зерновоз,Да,ТИХВИНС,Апрель 2019
3,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 101 м. куб.,...",19_9870_01,4.84,Хоппер-зерновоз,Да,ТИХВИНС,Апрель 2019
4,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 120 м. ку...,19_9549_03,4.84,Хоппер-минераловоз,Да,ТИХВИНС,Апрель 2019
5,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 101 м. куб.),19_9835_01,4.84,Хоппер-минераловоз,Да,ТИХВИНС,Апрель 2019
6,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-цементовоз (с осевой нагрузкой 25 тс),19_9550_01,4.84,хоппер-цементовоз,Да,ТИХВИНС,Апрель 2019
7,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",фитинговая 80-футовая платформа (с осевой нагр...,13_6903,4.64,Платформа фитинговая,Да,ТИХВИНС,Апрель 2019
8,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*",цистерна для перевозок нефтепродуктов (с осево...,15_9993,4.02,Цистерна д/нефтепродуктов,Да,ТИХВИНХ,Апрель 2019
9,Россия,"ТихвинХимМаш (млн. руб., франко-завод)*","цистерна для перевозок СУГ (сочлененного типа,...",15_9541_01,9.6,Цистерна СУГ. Сочлен,Да,ТИХВИНХ,Апрель 2019


In [15]:
# tb = df.copy()
# row1 = 0
# for models in tb['Модели']:
#     row2 = 0
#     for mod in wagon['Модель']:
#         if models == mod:
            
#             tb['Род вагона'][row1] = wagon['Тип'][row2]
#             tb['Инновационность'][row1] = wagon['Инновационность'][row2]
#             tb['Продукция'][row1] = wagon['Подробное описание'][row2]
#             if tb['Страна'][row1] == wagon['Страна'][row2]:
#                 tb['Производитель'][row1] = wagon['Завод'][row2]
#             else:
#                 tb['Производитель'][row1] = wagon.loc[(wagon['Модель'] == models) & (wagon['Страна'] == tb['Страна'][row1])]['Завод'].values[0]
#                 print ('Несоответствие страны в строке:',row1,'//', 'Страна выгрузки:', tb['Страна'][row1],'//','Страна в базе:',wagon['Страна'][row2] )
#         row2 = row2+1
#     row1 = row1 + 1
# tb['Сокращенное название (для связки со страной)'] = tb['Страна']    

In [16]:


# base = pd.read_excel('3. Цены на вагоны.xlsx', sheet_name='Цены база', skiprows=0, header=0)
# print(base.shape)
# base.head()
# base['Модели'] = base['Модели'].str.replace('_','-')
# base['Модели'] = base['Модели'].replace(' ','')
# df['Модели'] = df['Модели'].replace(' ','')
# df['Модели'][0]==wagon['Модель'][0]
# row = 0
# for model in df['Модели']:
#     df['Род вагона'][row] = base[base['Модели'] == model]['Продукция'].iloc[0]
    
# model = df['Модели'][0]
# base[base['Модели'] == model]['Продукция'].iloc[0]

# base[base['Модели'] == '19-9549']['Продукция'].iloc[0]

# model = df['Модели'][0]
# model

# df['Модели'][0]    
    

In [17]:

tb

,Страна,Производитель,Продукция,Модели,Цена,Род вагона,Месяц,Год,Инновационность,Сокращенное название (для связки со страной)
0,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с разгрузочными люками (с осевой наг...,12-9853,4.2,Полувагон,Март,2019,Да,ТИХВИНС
1,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",полувагон с глухим кузовом (с осевой нагрузкой...,12-9869,4.1,Полувагон,Март,2019,Да,ТИХВИНС
2,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 120 м. куб.,...",19-9549,4.41,Хоппер-зерновоз,Март,2019,Да,ТИХВИНС
3,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 126 м. куб.,...",19-9549-01,4.7,хоппер-зерновоз,Март,2019,Да,ТИХВИНС
4,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-зерновоз (с крышей из алюминиевого спла...,19-9549-02,4.7,хоппер-зерновоз,Март,2019,Да,ТИХВИНС
5,Россия,"Тихвинский вагоностроительный завод (млн. руб,...","хоппер-зерновоз (с объемом кузова 101 м. куб.,...",19-9870-01,4.7,Хоппер-зерновоз,Март,2019,Да,ТИХВИНС
6,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 120 м. ку...,19-9549-03,4.7,Хоппер-минераловоз,Март,2019,Да,ТИХВИНС
7,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-минераловоз (с объемом кузова 101 м. куб.),19-9835-01,4.7,Хоппер-минераловоз,Март,2019,Да,ТИХВИНС
8,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",хоппер-цементовоз (с осевой нагрузкой 25 тс),19-9550-01,4.7,хоппер-цементовоз,Март,2019,Да,ТИХВИНС
9,Россия,"Тихвинский вагоностроительный завод (млн. руб,...",фитинговая 80-футовая платформа (с осевой нагр...,13-6903,4.5,Фитинговая платформа,Март,2019,Да,ТИХВИНС
